In [3]:
import pandas as pd
import numpy as np
from IPython.display import display

print("--- START: Budowanie kompletnego słownika encji ---")

# --- Krok 1: Wczytanie oficjalnej bazy lokalizacji ---
try:
    df_meta_raw = pd.read_csv('lokalizacja.csv', sep=',', header=None, on_bad_lines='skip', low_memory=False)
    print("Plik 'lokalizacja.csv' wczytany pomyślnie.")
except FileNotFoundError:
    print("BŁĄD: Upewnij się, że plik 'lokalizacja.csv' znajduje się w folderze.")
    raise

# --- Krok 2: Nadanie nazw kolumnom i podstawowe czyszczenie ---
df_meta = df_meta_raw.copy()
df_meta.columns = ['LocationID', 'ParentId', 'Name', 'Type', 'Path']
df_meta.dropna(subset=['LocationID', 'Name'], inplace=True)
df_meta['LocationID'] = pd.to_numeric(df_meta['LocationID'], errors='coerce').fillna(0).astype(int).astype(str)
df_meta = df_meta[df_meta['LocationID'] != '0']
print(f"Wczytano {len(df_meta)} poprawnych wpisów z bazy lokalizacji.")


# --- Krok 3: Wyodrębnienie poszczególnych typów lokalizacji (POPRAWIONA WERSJA) ---
print("\n--- Wyodrębnianie encji na podstawie poprawnych typów ---")

# Używamy pełnych nazw typów, które zidentyfikowaliśmy w kroku diagnostycznym
# Dodajemy też inne popularne typy dróg do puli ulic (Al., Pl., Rondo)
df_powiaty = df_meta[df_meta['Type'] == 'Powiat'][['LocationID', 'Name']]
df_miasta = df_meta[df_meta['Type'] == 'Miasto'][['LocationID', 'Name']]
df_dzielnice = df_meta[df_meta['Type'] == 'Dzielnica'][['LocationID', 'Name']]
df_ulice = df_meta[df_meta['Type'].isin(['Ul.', 'Al.', 'Pl.', 'Rondo', 'Skwer'])][['LocationID', 'Name']]


print(f"\nZnaleziono encji w bazie:")
print(f" - Powiaty: {len(df_powiaty)}")
print(f" - Miasta: {len(df_miasta)}")
print(f" - Dzielnice: {len(df_dzielnice)}")
print(f" - Ulice (i podobne): {len(df_ulice)}")

# --- Krok 4: Ujednolicenie i dodanie słownika dla BuildingType ---
building_type_dict = {
    '-1': 'blok', '-2': 'kamienica', '-3': 'apartamentowiec',
    '-4': 'dom', '-5': 'plomba', '-99': 'inny'
}
df_building_types = pd.DataFrame(list(building_type_dict.items()), columns=['LocationID', 'Name'])
print(f" - Typy budynków (zdefiniowane ręcznie): {len(df_building_types)}")


# --- Krok 5: Połączenie wszystkich słowników w jeden kompletny ---
df_slownik_kompletny = pd.concat([
    df_powiaty, df_miasta, df_dzielnice, df_ulice, df_building_types
], ignore_index=True)
df_slownik_kompletny.drop_duplicates(subset='LocationID', keep='first', inplace=True)
print(f"\nZbudowano finalny, kompletny słownik z {len(df_slownik_kompletny)} unikalnych wpisów.")

# --- Krok 6: Zapis i weryfikacja ---
if not df_slownik_kompletny.empty:
    output_filename = 'slownik_kompletny.csv'
    print(f"Zapisywanie słownika do pliku: '{output_filename}'...")
    df_slownik_kompletny.to_csv(output_filename, index=False, sep=';')
    print("Zapis zakończony sukcesem!")

    print("\nPróbka 20 losowych wierszy z finalnego słownika:")
    sample_size = min(20, len(df_slownik_kompletny))
    if sample_size > 0:
        display(df_slownik_kompletny.sample(sample_size))
else:
    print("Słownik jest pusty. Coś poszło nie tak.")

--- START: Budowanie kompletnego słownika encji ---
Plik 'lokalizacja.csv' wczytany pomyślnie.
Wczytano 398832 poprawnych wpisów z bazy lokalizacji.

--- Wyodrębnianie encji na podstawie poprawnych typów ---

Znaleziono encji w bazie:
 - Powiaty: 314
 - Miasta: 913
 - Dzielnice: 18
 - Ulice (i podobne): 289194
 - Typy budynków (zdefiniowane ręcznie): 6

Zbudowano finalny, kompletny słownik z 290445 unikalnych wpisów.
Zapisywanie słownika do pliku: 'slownik_kompletny.csv'...
Zapis zakończony sukcesem!

Próbka 20 losowych wierszy z finalnego słownika:


,LocationID,Name
59776,317952,Równa
239773,500335,Wiejska
98621,357314,Słoneczna
65280,323530,Zdrojowa
248304,508968,Laskowa
98658,357351,Cedrowa
56238,314367,Krótka
125740,384794,Jeziorna
105094,363871,Sopelkowa
103444,362206,Konwaliowa
